# Improv casting optimization
### Using Integer Programming to find casting assignments that maximize group happiness

In [246]:
import pulp as pl
import pandas as pd

In [247]:
def get_skits(df):
    return df.columns[1:].tolist()

def get_players(df):
    return df["Player"].tolist()
    

def validate_player_ranks(df):
    # Validate that every player has ranked the skits in correct numerical order
    # Just a bs check to make sure we don't have like 3 5s in one persons row for some reason
    skits = get_skits(df)
    expected_ranks = set(range(1, len(skits) + 1))

    # Check all players at once
    invalid_mask = df[skits].apply(lambda row: set(row) != expected_ranks, axis=1)

    # List invalid players
    invalid_players = df.loc[invalid_mask, "Player"].tolist()

    if invalid_players:
        raise ValueError(f"Players with invalid ranks: {invalid_players}")
    else:
        print("All players have valid rankings!")

In [248]:
# Read in player rankings. Assumes rows are players, columns are games
# Assume players rank things from 1 (most desired) to n_skits (least desired)
df = pd.read_csv("cast_preferences.csv")
validate_player_ranks(df)
df.head()

All players have valid rankings!


,Player,4 corners,New choice,Alphabet,Nightclub,3579,Host
0,Alex,3,2,1,5,4,6
1,Rachel,1,2,3,4,5,6
2,Senoll,6,3,1,4,5,2
3,Ashley,2,6,4,1,3,5
4,Owen,4,3,6,2,5,1


In [249]:
# Players and skits
players = df["Player"].tolist()
skits = df.columns[1:].tolist()

player_ids = range(len(players))
skit_ids = range(len(skits))

In [ ]:
# Fairness coefficient can be toggled to upweight/downweight the importance of the least happy person
# FC == 0: Maximize total group happiness, even if some people get fucked
# FC <= 5: Mostly maximize total group happiness, but reduce the chance someone is getting fucked
# FC >= 1000: Maximize fairness even if the group as a whole is less happy
fairness_coefficient = 5

# Whether host can appear in the games
host_plays_games = True


In [251]:
# Max happiness value
max_happiness = int(df.max(numeric_only=True).max())

# Invert the player ranks matrix into a player happiness matrix
# Happiness == max_rank - rank (e.g., 6 - 1 = 5 happiness; 6 - 6 = 0 happiness)
player_happiness = df[skits].applymap(lambda r: max_happiness - r).values.tolist()

<positron-console-cell-251>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [252]:
# --- CUSTOM SKIT SIZES ---
# Exact number of players needed for each skit. Adjust as needed. 
skit_sizes = {
    "4 corners": 4,
    "New choice": 3,
    "Alphabet": 3,
    "Nightclub": 7,
    "3579": 4,
    "Host": 1
}

# Make sure skits are exactly as they appear in the csv
assert sorted(list(skit_sizes.keys())) == sorted(skits), "Skits sizes doesn't match names of skits in input CSV"

### Create base model

In [253]:
# Initialize model object
model = pl.LpProblem("ImprovCasting", pl.LpMaximize)

# Create variables to represent the casting matrix (i.e., which players are in each skit)
# These variables will be permuted in order to maximize the happiness of the couch
cast = pl.LpVariable.dicts("cast", (player_ids, skit_ids), lowBound=0, upBound=1, cat="Binary")

# Minimum happiness
min_happiness = pl.LpVariable("min_happiness", lowBound=0)

# This is the function whose value we're trying to optimize
# Each cast permutation will be scored by multiplying cast matrix * player_happiness matrix
# Finally, we add a term that multiplies the least happy players happiness by the happiness_coefficient

model += pl.lpSum(player_happiness[p][s] * cast[p][s] for p in player_ids for s in skit_ids) + fairness_coefficient * min_happiness

### Add model constraints

In [254]:
# Ensure each player appears in 1 or 2 skits
for player in player_ids:
    model += pl.lpSum(cast[player][skit] for skit in skit_ids) >= 1
    model += pl.lpSum(cast[player][skit] for skit in skit_ids) <= 2

for skit_id, skit_name in enumerate(skits):
    model += pl.lpSum(cast[player][skit_id] for player in player_ids) == skit_sizes[skit_name]

# Enforce custom skit sizes
for skit, skit_name in enumerate(skits):
    model += pl.lpSum(cast[player][skit] for player in player_ids) == skit_sizes[skit_name]

# Fairness: every player's happiness >= min_happiness
for player in player_ids:
    model += pl.lpSum(
        player_happiness[player][skit] * cast[player][skit] for skit in skit_ids
    ) >= min_happiness

# Conditionally add constraint that host can't play games
if not host_plays_games:
    # Find the skit_id for "Host"
    host_id = skits.index("Host")

    # Host constraint: if a player is in Host, they can't be in any other skit
    # If player is Host (cast[p][host_id] = 1) -> they cannot be in any other skit.
    # If player is not Host (cast[p][host_id] = 0) -> they may be in up to 2 other skits.
    for p in player_ids:
        model += pl.lpSum(cast[p][s] for s in skit_ids if s != host_id) <= 2 * (1 - cast[p][host_id])

In [255]:
# Solve the model
model.solve(pl.PULP_CBC_CMD(msg=False))

1

In [256]:
status = pl.LpStatus[model.status]
print(f"Solver status: {status}")

Solver status: Optimal


In [257]:
# Create a list of (player, skit) tuples instead
assignments = [(players[p_id], skits[s_id]) 
               for p_id in player_ids 
               for s_id in skit_ids 
               if pl.value(cast[p_id][s_id]) > 0.5]

# Then group by skit
print("Casting results:")
for skit in skits:
    cast_list = [player for player, assigned_skit in assignments if assigned_skit == skit]
    print(f"{skit} ({skit_sizes[skit]} slots): {', '.join(cast_list)} (count: {len(cast_list)})")

print("\n")
print("Skits by person:")
for p in players:
    game_list = [assigned_skit for player, assigned_skit in assignments if player == p]
    print(f"{p}: {', '.join(game_list)}")


Casting results:
4 corners (4 slots): Rachel, Andre, Mike, Maddie (count: 4)
New choice (3 slots): Alex, Mike, Fred (count: 3)
Alphabet (3 slots): Alex, Senoll, Adam (count: 3)
Nightclub (7 slots): Rachel, Senoll, Ashley, Annalise, Sita, Adam, Fred (count: 7)
3579 (4 slots): Ashley, Annalise, Sita, Maddie (count: 4)
Host (1 slots): Owen (count: 1)


Skits by person:
Alex: New choice, Alphabet
Rachel: 4 corners, Nightclub
Senoll: Alphabet, Nightclub
Ashley: Nightclub, 3579
Owen: Host
Andre: 4 corners
Annalise: Nightclub, 3579
Mike: 4 corners, New choice
Sita: Nightclub, 3579
Maddie: 4 corners, 3579
Adam: Alphabet, Nightclub
Fred: New choice, Nightclub
